In [2]:
import torch

from torch import nn, optim

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [7]:
class MutiLayerPerceptronModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(2, 10),
            nn.Sigmoid(), 
            nn.Linear(10, 10), 
        
            nn.Sigmoid(), 
            nn.Linear(10, 10), 
            
            nn.Sigmoid(), 
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.layer(x)
    
# model = nn.Sequential(
#             nn.Linear(2, 10),
#             nn.Sigmoid(),
#             nn.Linear(10, 10),

#             nn.Sigmoid(),
#             nn.Linear(10, 10),

#             nn.Sigmoid(),
#             nn.Linear(10, 1),
#             nn.Sigmoid()
#         ).to(device)

In [8]:
model = MutiLayerPerceptronModel()
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1)

In [10]:
for epoch in range(7801):
    
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print('Epoch {:05d}, Cost: {:.6f}'.format(epoch, cost.item()))

Epoch 00000, Cost: 0.000315
Epoch 01000, Cost: 0.000217
Epoch 02000, Cost: 0.000164
Epoch 03000, Cost: 0.000132
Epoch 04000, Cost: 0.000110
Epoch 05000, Cost: 0.000094
Epoch 06000, Cost: 0.000082
Epoch 07000, Cost: 0.000072


In [11]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('Hypothesis :', hypothesis.detach().cpu().numpy())
    print('Predicted :', predicted.detach().cpu().numpy())
    print('Y :', Y.cpu().numpy())
    print('Accuracy :', accuracy.item())

Hypothesis : [[4.7401376e-05]
 [9.9992716e-01]
 [9.9993300e-01]
 [7.7828867e-05]]
Predicted : [[0.]
 [1.]
 [1.]
 [0.]]
Y : [[0.]
 [1.]
 [1.]
 [0.]]
Accuracy : 1.0


In [12]:
from torch.utils.data import Dataset, DataLoader

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed(1)

In [14]:
class TestDataset(Dataset):
    def __init__(self):
        self.x_data = torch.FloatTensor([[0, 0], 
                                         [0, 1], 
                                         [1, 0], 
                                         [1, 1]]).to(device)
        self.y_data = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

In [15]:
class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(2, 10),
            nn.Sigmoid(),
            nn.Linear(10, 10),

            nn.Sigmoid(),
            nn.Linear(10, 10),

            nn.Sigmoid(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layer(x)

In [16]:
dataset = TestDataset()
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [17]:
model = TestModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1)

In [18]:
nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples

        optimizer.zero_grad()
        hypothesis = model(x_train)
        cost = criterion(hypothesis, y_train)
        cost.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print('Epoch {:5d}/{} Batch {}/{} Cost: {:.4f}'.format(
                epoch, nb_epochs, batch_idx + 1, len(dataloader), cost.item()))

Epoch     0/10000 Batch 1/1 Cost: 0.7319
Epoch   100/10000 Batch 1/1 Cost: 0.6931
Epoch   200/10000 Batch 1/1 Cost: 0.6931
Epoch   300/10000 Batch 1/1 Cost: 0.6931
Epoch   400/10000 Batch 1/1 Cost: 0.6931
Epoch   500/10000 Batch 1/1 Cost: 0.6931
Epoch   600/10000 Batch 1/1 Cost: 0.6931
Epoch   700/10000 Batch 1/1 Cost: 0.6931
Epoch   800/10000 Batch 1/1 Cost: 0.6931
Epoch   900/10000 Batch 1/1 Cost: 0.6931
Epoch  1000/10000 Batch 1/1 Cost: 0.6931
Epoch  1100/10000 Batch 1/1 Cost: 0.6931
Epoch  1200/10000 Batch 1/1 Cost: 0.6931
Epoch  1300/10000 Batch 1/1 Cost: 0.6931
Epoch  1400/10000 Batch 1/1 Cost: 0.6931
Epoch  1500/10000 Batch 1/1 Cost: 0.6931
Epoch  1600/10000 Batch 1/1 Cost: 0.6931
Epoch  1700/10000 Batch 1/1 Cost: 0.6931
Epoch  1800/10000 Batch 1/1 Cost: 0.6931
Epoch  1900/10000 Batch 1/1 Cost: 0.6931
Epoch  2000/10000 Batch 1/1 Cost: 0.6931
Epoch  2100/10000 Batch 1/1 Cost: 0.6931
Epoch  2200/10000 Batch 1/1 Cost: 0.6931
Epoch  2300/10000 Batch 1/1 Cost: 0.6931
Epoch  2400/1000

In [19]:
with torch.no_grad():
    hypothesis = model(x_train)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == y_train).float().mean()
    print('Hypothesis :', hypothesis.detach().cpu().numpy())
    print('Predicted :', predicted.detach().cpu().numpy())
    print('Y :', y_train.cpu().numpy())
    print('Accuracy :', accuracy.item())

Hypothesis : [[1.1827720e-04]
 [9.9986887e-01]
 [9.9983013e-01]
 [1.6409562e-04]]
Predicted : [[0.]
 [1.]
 [1.]
 [0.]]
Y : [[0.]
 [1.]
 [1.]
 [0.]]
Accuracy : 1.0
